# Service 프로그래밍(python)

### 코드분석
+ 빨간 로봇을 등장시키는 client는 py_service_pkg에서 spawn_model.py이였는데 코드는 밑에 나와있다.
<pre>
<code>
import os

from ament_index_python.packages import get_package_share_directory
from gazebo_msgs.srv import SpawnEntity
import rclpy
from rclpy.node import Node


class SpawnRobot(Node):

    def __init__(self):
        super().__init__('gazebo_model_spawner')
        self.client = self.create_client(SpawnEntity, '/spawn_entity')

        while not self.client.wait_for_service(timeout_sec=1.0):
            self.get_logger().error('service not available, waiting again...')

        # Get urdf path
        self.urdf_file_path = os.path.join(
            get_package_share_directory('gcamp_gazebo'),
            'urdf',
            'skidbot2.urdf',
        )

        self.req = SpawnEntity.Request()

    def send_req(self):
        self.req.name = 'skidbot2'
        self.req.xml = open(self.urdf_file_path, 'r').read()
        self.req.robot_namespace = 'skidbot2'
        self.req.initial_pose.position.x = 1.0
        self.req.initial_pose.position.y = 1.0
        self.req.initial_pose.position.z = 0.3

        self.get_logger().debug('==== Sending service request to `/spawn_entity` ====')
        self.future = self.client.call_async(self.req)

        return self.future


def main(args=None):

    rclpy.init(args=args)

    robot_spawn_node = SpawnRobot()
    future = robot_spawn_node.send_req()

    rclpy.spin_until_future_complete(robot_spawn_node, future)

    if future.done():
        try:
            response = future.result()
        except Exception:
            raise RuntimeError(
                'exception while calling service: %r' % future.exception()
            )
        else:
            robot_spawn_node.get_logger().info('==== Service Call Done ====')
            robot_spawn_node.get_logger().info(f'Status_message : {response.status_message}')
        finally:
            robot_spawn_node.get_logger().warn('==== Shutting down node. ====')
            robot_spawn_node.destroy_node()
            rclpy.shutdown()


if __name__ == '__main__':
    main()



+ import부분 분석
<pre>
<code>
import os

from ament_index_python.packages import get_package_share_directory
from gazebo_msgs.srv import SpawnEntity #이 코드를 통해서 srv타입 결정
import rclpy
from rclpy.node import Node
</code>
</pre>
+ from gazebo_msgs.srv import SpawnEntity를 통해서 srv타입 결정한다.
    
    + $ ros2 interface show gazebo_msgs/srv/SpawnEntity  
    string name                         # Name of the entity to be spawned (optional).  
    string xml                          # Entity XML description as a string, either URDF or SDF.  
    string robot_namespace              # Spawn robot and all ROS interfaces under this namespace  
    geometry_msgs/Pose initial_pose     # Initial entity pose.  
    string reference_frame              # initial_pose is defined relative to the frame of this entity.  

    ---
    bool success                      # Return true if spawned successfully.  
    string status_message             # Comments if available.  

    + 을 통해서 위의 5개 request를 모두 만족할때 모델을 등장시킬 수 있다.
    
+ get_package_share_directory는 ROS2 package의 file system에 접근하기 위해서 필요한 코드이다.

### Class 내부 분석

<pre>
<code>
class SpawnRobot(Node):

    def __init__(self):
        super().__init__('gazebo_model_spawner')
        self.client = self.create_client(SpawnEntity, '/spawn_entity')

        while not self.client.wait_for_service(timeout_sec=1.0):
            self.get_logger().error('service not available, waiting again...')

        # Get urdf path
        self.urdf_file_path = os.path.join(
            get_package_share_directory('gcamp_gazebo'),
            'urdf',
            'skidbot2.urdf',
        )

        self.req = SpawnEntity.Request()

        </code>
        </pre>
+ Client를 생성(Service srv타입, Service 서버 자체의 이름을 통해서)하고 오타가 났을 경우를 대비하여 wait_for_service라는 함수가 내장되어 있다.
    + '/spawn_entity'(Service 서버 자체의 이름) 이 Server가 지정한 Service이름과 일치해야만 제대로 request를 보낼 수 있다.
+ Clinet는 request에만 접근하기 때문에 SpawnEntity.Request()처럼 request에만 따로 접근할 수 있도록 나뉘어져 있다.

### 실질적으로 request를 보내는 부분
<pre>
<code>
    def send_req(self):
        self.req.name = "skidbot2"
        self.req.xml = open(self.urdf_file_path, "r").read()
        self.req.robot_namespace = "skidbot2"
        self.req.initial_pose.position.x = 1.0
        self.req.initial_pose.position.y = 1.0
        self.req.initial_pose.position.z = 0.3

        print("==== Sending service request to `/spawn_entity` ====")
        self.future = self.client.call_async(self.req)

        return self.future
</code>
</pre>
  + self.req.xml = open(self.urdf_file_path, "r").read()에서 xml이 실제 modeling file이 들어갈 부분인데 open을 이용하여 read를 통해 읽을 것 이라고 지정해주면 거대한 xml 파일의 내용을 그대로 가져올 수 있다. 
  + self.req.robot_namespace = "skidbot2"를 통해 새로 생긴 로봇을 운전하기 위해서 namespace에 이름(skidbot2)을 지정해준다.
  + self.req.initial_pose.position.z = 0.3을 통해 로봇이 등장할때 바닥에 박히는 것을 방지한다.
  + call_async는 실질적으로 request를 보내는 함수이다.
    + call_async함수가 반환하는 것 중에 future라는 것이 존재한다.
      + future는 특정 작업에 대한 약속을 받아내는 것이다.
      + future.result를 통해서 response를 가져올 수 있다.
    
+ 결국 server는 request를 보내고 reponse를 기다리고 그 response를 확인하는 작업을 한다고 볼 수 있다.

### Turning Server Client
<pre>
<code>
# Terminal 1
$ rosfoxy 
$ ros2 launch gcamp_gazebo gcamp_world.launch.py

# Terminal 2 Service Server
$ rosfoxy
$ ros2 run py_service_pkg robot_turning_server

# Terminal 3 Service Client Call
$ rosfoxy
$ ros2 run py_service_pkg robot_turning_client
[INFO] [1626791127.915754676] [robot_turn_client]: ==== Robot Turn Service Client ====
> Type turning time duration: 5
> Type turning linear velocity: 0.5
> Type turning angular velocity: 1.0
[INFO] [1626791138.941500026] [robot_turn_client]: linear_x : 0.5 / angular_z : 1.0
[INFO] [1626791138.941818373] [robot_turn_client]:  Request Sended 
[INFO] [1626791143.001586224] [robot_turn_client]: ==== Service Call Done ====
[INFO] [1626791143.001916901] [robot_turn_client]: Result Message : Success
[WARN] [1626791143.002255509] [robot_turn_client]: ==== Shutting down node ====
</code>
</pre>
+ Service를 통해 실행시키는 것을 확인할 수 있다.
+ Client를 2개로 설정하여 Service의 특성을 확인할 수 있다.(첫번째 Client가 끝날때까지 두번째 Client가 대기한다.)


### 직접 srv 만들어보기

### Service Server 작성
+ Service 생성
    + create_service를 통해서 생성한다.
    + 사용되는 srv타입, service이름, request가 들어올 때마다 실행될 callback의 3개의 매개변수를 받는다.
    <pre>
    <code>
    def robot_turn_callback(self, request, response):
        self.start_time = self.get_clock().now().to_msg().sec

				# move_robot은 topic 시간에 살펴본 바와 동일합니다.
        self.move_robot(
            request.time_duration, request.linear_vel_x, request.angular_vel_z
        )
        self.stop_robot()
			
				# response를 채워줍니다.
        response.success = True
        self.get_logger().info("Servie Process Done...")

        return response
    </code>
    </pre>
    
  + callback내부를 살펴보면{def robot_turn_callback(self, request, response):}무엇이 왔는지 알고 logic을 통해 처리하고 정보를 주기 위해 request와 reponse를 받는 것을 확인할 수 있다.

        


+ main문
    <pre>
    <code>
    def main(args=None):
    rclpy.init(args=args)

    robot_turn_server = RobotTurnServer()

    rclpy.spin(robot_turn_server)

    robot_turn_server.destroy_node()
    rclpy.shutdown()
    </code>
    </pre>
    + main문은 spin으로 되어 있어서 위 과정들이 무한해서 반복된다.